In [0]:
import json
import transformers
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import mlflow
import time
import datetime
# import keras
import nltk
from nltk.tokenize.util import align_tokens
from transformers import get_linear_schedule_with_warmup
nltk.download('punkt')
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset,Dataset ,DataLoader, RandomSampler, SequentialSampler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [0]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [0]:
#create dataframe from data
def read_file(path):
  data=open(path, "r")
  data=json.load(data)
  # print(data)
  string_labels = []
  for item in data:
    text = item['Text']
    id= item['id']
    sentences_split = nltk.word_tokenize(item["Text"])
    # print(sentences_split)
    try:
        token_spans = align_tokens(sentences_split,text)
    except:
        print("Error")
        # print(token_spans)
    for i in range(len(token_spans)):
        token_in_annotation = False
        for annotation in item['annotation']:
            if int(annotation["start"])<= token_spans[i][0] and int(annotation['end'])>=token_spans[i][1]:
                string_labels.append((text[token_spans[i][0]:token_spans[i][1]],annotation['type']))
                token_in_annotation = True
        if(token_in_annotation==False):
            string_labels.append((text[token_spans[i][0]:token_spans[i][1]],'O'))
  df = pd.DataFrame(string_labels, columns =['token', 'label'])
  return df

In [0]:
path="/dbfs/FileStore/Chanchal/Data_json/gene_protein.json"
train_df=read_file(path)



Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:

path="/dbfs/FileStore/Chanchal/Data_json/biored.json"
test_df=read_file(path)



Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:
target_labels = ['GP', 'GeneOrGeneProduct', 'protein','O']
train_df = train_df[train_df['label'].isin(target_labels)]
train_df['label'] = train_df['label'].replace(['GP','GeneOrGeneProduct', 'protein'], 'Gene_protein')
print("Number of tags: {}".format(len(train_df.label.unique())))
frequencies = train_df.label.value_counts()
frequencies

Number of tags: 2


O               932164
Gene_protein    126717
Name: label, dtype: int64

In [0]:

test_df = test_df[test_df['label'].isin(target_labels)]
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 2


O                    141466
GeneOrGeneProduct      8168
Name: label, dtype: int64

In [0]:
# Aggregating tokens into a sentence to be used as input for training data
def aggr(df):
  df['group']=(df['token'] == '.').cumsum()
  new_df = df.groupby('group').agg({'token': ' '.join, 'label': ' '.join}).reset_index(drop=True)
  new_df.rename(columns = {'token':'text', 'label':'labels'}, inplace = True)
  return new_df

In [0]:
train_n_df=aggr(train_df)
test_n_df=aggr(test_df)

In [0]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in train_n_df['labels'].values.tolist()]

# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

In [0]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'Gene_protein': 0, 'O': 1}


In [0]:

MAX_LEN = 512
batch_size=16
epochs = 3
lr = 2e-04
MAX_GRAD_NORM = 10

In [0]:
# Tokenizing the input
def tokenizing(text):
  txt=text.values.tolist()
  text_tokenized = tokenizer(txt, padding='max_length', max_length=MAX_LEN , truncation=True, return_tensors="pt")
  input_ids=text_tokenized['input_ids']
  attention_masks=text_tokenized['attention_mask']
  return input_ids,attention_masks

In [0]:
#Balancing the labels according to the tokens
def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=MAX_LEN , truncation=True)
    label_all_tokens=True

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx


    return label_ids

In [0]:
#converting text and labels for train_data

lb = [i.split() for i in train_n_df['labels'].values.tolist()]
txt = train_n_df['text'].values.tolist()
train_new_label=[align_label(i,j) for i,j in zip(txt, lb)]
train_input_ids,train_attention_masks=tokenizing(train_n_df['text'])

In [0]:
#converting text and labels for test_data

lb = [i.split() for i in test_n_df['labels'].values.tolist()]
txt = test_n_df['text'].values.tolist()
test_new_label=[align_label(i,j) for i,j in zip(txt, lb)]
test_input_ids,test_attention_masks=tokenizing(test_n_df['text'])

In [0]:
train_pt_input_ids = torch.stack(list(train_input_ids), dim=0)

train_pt_attention_masks = torch.stack(list(train_attention_masks), dim=0)

train_pt_labels = torch.tensor(train_new_label, dtype=torch.long)

In [0]:
test_pt_input_ids = torch.stack(list(test_input_ids), dim=0)

test_pt_attention_masks = torch.stack(list(test_attention_masks), dim=0)

test_pt_labels = torch.tensor(test_new_label, dtype=torch.long)

In [0]:
from torch.utils.data import TensorDataset, random_split
import random
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_pt_input_ids, train_pt_attention_masks, train_pt_labels)
test_dataset= TensorDataset(test_pt_input_ids, test_pt_attention_masks, test_pt_labels)


train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

test_size=len(test_dataset)
train_dataset, valid_dataset = random_split(train_dataset, [train_size, val_size])


print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))

34,736 training samples
8,684 validation samples
6,369 testing samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size )

validation_dataloader = DataLoader(valid_dataset, sampler = SequentialSampler(valid_dataset), batch_size = batch_size)

test_dataloader=DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size )


In [0]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

from transformers import BertForTokenClassification, AdamW, BertConfig


model = BertForTokenClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",num_labels=len(labels_to_ids))
# model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))
model.cuda()

In [0]:
# defining the optimizer

optimizer = AdamW(model.parameters(),
                  lr)


# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [0]:

mlflow.start_run()
loss_values = []
val_loss_vaues=[]
total_t0 = time.time()
for epoch in range(epochs):
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
  print('Training...')
  # Measure how long the training epoch takes.
  training_stats = []
  model.train()
  total_loss = 0
  t0 = time.time()
  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    optimizer.zero_grad()
    loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).to_tuple()
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

  avg_train_loss = total_loss / len(train_dataloader)
  loss_values.append(avg_train_loss)
  # Measure how long this epoch took.
  training_time = format_time(time.time() - t0)
  
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  # print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(training_time))
  # Validation
  print("Running Validation...")


  t0 = time.time()



  model.eval()
  total_eval_loss = 0
  val_predictions = []
  for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
      loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,labels=b_labels).to_tuple()
    total_eval_loss += loss.item()
    predicted_labels = torch.argmax(logits, dim=2)
    val_predictions.extend(predicted_labels.detach().cpu().numpy())

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  val_loss_vaues.append(avg_val_loss)
  validation_time = format_time(time.time() - t0)

  print("  Average validation loss: {0:.2f}".format(avg_val_loss))
  mlflow.log_param("lr", lr)
  mlflow.log_metric("epoch", epoch + 1)
  mlflow.log_metric("average_train_loss", avg_train_loss, step=epoch+1)
  mlflow.log_metric("average_val_loss", avg_val_loss,step=epoch+1)
  
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

mlflow.end_run()

======== Epoch 1 / 3 ========
Training...
  Batch    40  of  2,171.    Elapsed: 0:00:55.
  Batch    80  of  2,171.    Elapsed: 0:01:50.
  Batch   120  of  2,171.    Elapsed: 0:02:47.
  Batch   160  of  2,171.    Elapsed: 0:03:46.
  Batch   200  of  2,171.    Elapsed: 0:04:44.
  Batch   240  of  2,171.    Elapsed: 0:05:41.
  Batch   280  of  2,171.    Elapsed: 0:06:39.
  Batch   320  of  2,171.    Elapsed: 0:07:36.
  Batch   360  of  2,171.    Elapsed: 0:08:34.
  Batch   400  of  2,171.    Elapsed: 0:09:32.
  Batch   440  of  2,171.    Elapsed: 0:10:30.
  Batch   480  of  2,171.    Elapsed: 0:11:28.
  Batch   520  of  2,171.    Elapsed: 0:12:22.
  Batch   560  of  2,171.    Elapsed: 0:13:14.
  Batch   600  of  2,171.    Elapsed: 0:14:07.
  Batch   640  of  2,171.    Elapsed: 0:15:02.
  Batch   680  of  2,171.    Elapsed: 0:15:58.
  Batch   720  of  2,171.    Elapsed: 0:16:56.
  Batch   760  of  2,171.    Elapsed: 0:17:54.
  Batch   800  of  2,171.    Elapsed: 0:18:49.
  Batch   840  of 

In [0]:
# print('Predicting labels for {:,} test sentences...'.format(len(pt_input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # Telling the model not to compute or store gradients, saving memory and 

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [0]:
from sklearn.metrics import f1_score

# First, combine the results across the batches.
all_predictions = np.concatenate(predictions, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

print("After flattening the batches, the predictions have shape:")
print("    ", all_predictions.shape)

# Next, let's remove the third dimension (axis 2), which has the scores
# for all 18 labels. 

# For each token, pick the label with the highest score.
predicted_label_ids = np.argmax(all_predictions, axis=2)

print("\nAfter choosing the highest scoring label for each token:")
print("    ", predicted_label_ids.shape) 


# Eliminate axis 0, which corresponds to the sentences.
predicted_label_ids = np.concatenate(predicted_label_ids, axis=0)
all_true_labels = np.concatenate(all_true_labels, axis=0)

print("\nAfter flattening the sentences, we have predictions:")
print("    ", predicted_label_ids.shape)
print("and ground truth:")
print("    ", all_true_labels.shape)

After flattening the batches, the predictions have shape:
     (6369, 512, 2)

After choosing the highest scoring label for each token:
     (6369, 512)

After flattening the sentences, we have predictions:
     (3260928,)
and ground truth:
     (3260928,)


In [0]:
# Construct new lists of predictions which don't include any null tokens.
real_token_predictions = []
real_token_labels = []

# For each of the input tokens in the dataset...
for i in range(len(all_true_labels)):

    # If it's not a token with a null label...
    if not all_true_labels[i] == -100:
        
        # Add the prediction and the ground truth to their lists.
        real_token_predictions.append(predicted_label_ids[i])
        real_token_labels.append(all_true_labels[i])

print("Before filtering out `null` tokens, length = {:,}".format(len(all_true_labels)))
print(" After filtering out `null` tokens, length = {:,}".format(len(real_token_labels)))

Before filtering out `null` tokens, length = 3,260,928
 After filtering out `null` tokens, length = 146,314


In [0]:
from sklearn.metrics import f1_score
# from seqeval.metrics import classification_report


# f1 = f1_score(real_token_labels, real_token_predictions, average='micro')
labels = [ids_to_labels[id.item()] for id in real_token_labels]
predictions = [ids_to_labels[id.item()] for id in real_token_predictions]

# print ("F1 score: {:.2%}".format(f1))

In [0]:
# from seqeval.metrics import classification_report
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

Gene_protein       0.00      0.00      0.00         0
           O       1.00      0.99      1.00    146314

    accuracy                           0.99    146314
   macro avg       0.50      0.50      0.50    146314
weighted avg       1.00      0.99      1.00    146314



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
report = classification_report(labels, predictions, output_dict=True)

df = pd.DataFrame(report).transpose()

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
df.to_csv("/dbfs/FileStore/Chanchal/logs/pubmedbertgene(all)-biored1.csv", index = True)

In [0]:

#Read the data
path="/dbfs/FileStore/Chanchal/Data_json/JNPLBA.json"
test_df=read_file(path)


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:
#addding data to dataframe

target_labels = ['protein', 'O']
test_df = test_df[test_df['label'].isin(target_labels)]
test_df['label'] = test_df['label'].replace(['protein'], 'Gene_protein')
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 2


O               207324
Gene_protein     28195
Name: label, dtype: int64

In [0]:
test_n_df=aggr(test_df)

In [0]:

#converting text and labels for test_data

lb = [i.split() for i in test_n_df['labels'].values.tolist()]
txt = test_n_df['text'].values.tolist()
test_new_label=[align_label(i,j) for i,j in zip(txt, lb)]
test_input_ids,test_attention_masks=tokenizing(test_n_df['text'])

In [0]:
test_pt_input_ids = torch.stack(list(test_input_ids), dim=0)

test_pt_attention_masks = torch.stack(list(test_attention_masks), dim=0)

test_pt_labels = torch.tensor(test_new_label, dtype=torch.long)

In [0]:

test_dataset= TensorDataset(test_pt_input_ids, test_pt_attention_masks, test_pt_labels)

test_dataloader=DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size )

In [0]:
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # Telling the model not to compute or store gradients, saving memory and 

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [0]:
# First, combine the results across the batches.
all_predictions = np.concatenate(predictions, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

print("After flattening the batches, the predictions have shape:")
print("    ", all_predictions.shape)

# Next, let's remove the third dimension (axis 2), which has the scores
# for all 18 labels. 

# For each token, pick the label with the highest score.
predicted_label_ids = np.argmax(all_predictions, axis=2)

print("\nAfter choosing the highest scoring label for each token:")
print("    ", predicted_label_ids.shape) 


# Eliminate axis 0, which corresponds to the sentences.
predicted_label_ids = np.concatenate(predicted_label_ids, axis=0)
all_true_labels = np.concatenate(all_true_labels, axis=0)

print("\nAfter flattening the sentences, we have predictions:")
print("    ", predicted_label_ids.shape)
print("and ground truth:")
print("    ", all_true_labels.shape)

After flattening the batches, the predictions have shape:
     (9987, 512, 2)

After choosing the highest scoring label for each token:
     (9987, 512)

After flattening the sentences, we have predictions:
     (5113344,)
and ground truth:
     (5113344,)


In [0]:
# Construct new lists of predictions which don't include any null tokens.
real_token_predictions = []
real_token_labels = []

# For each of the input tokens in the dataset...
for i in range(len(all_true_labels)):

    # If it's not a token with a null label...
    if not all_true_labels[i] == -100:
        
        # Add the prediction and the ground truth to their lists.
        real_token_predictions.append(predicted_label_ids[i])
        real_token_labels.append(all_true_labels[i])

print("Before filtering out `null` tokens, length = {:,}".format(len(all_true_labels)))
print(" After filtering out `null` tokens, length = {:,}".format(len(real_token_labels)))

Before filtering out `null` tokens, length = 5,113,344
 After filtering out `null` tokens, length = 245,075


In [0]:
labels = [ids_to_labels[id.item()] for id in real_token_labels]
predictions = [ids_to_labels[id.item()] for id in real_token_predictions]

In [0]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

Gene_protein       0.86      0.92      0.89     30752
           O       0.99      0.98      0.98    214323

    accuracy                           0.97    245075
   macro avg       0.92      0.95      0.94    245075
weighted avg       0.97      0.97      0.97    245075



In [0]:
report = classification_report(labels, predictions, output_dict=True)

df2 = pd.DataFrame(report).transpose()

In [0]:
df2.to_csv("/dbfs/FileStore/Chanchal/logs/pubmedbertgene(all)--jnplba1.csv", index = True)